<a href="https://colab.research.google.com/github/Violeta759/Clase5/blob/main/Tree_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Con **JUPITER** puedes editar los colabs y presentar too.

# Árboles de decisión.
Algortimos supervisados (datos y clase).

Sea una colección de datos $x_i \in R^n$ para $i=0,\ldots,l$ y un vector de etiquetas $y \in R^l$.

Consideremos $Q$ los datos del nodo $m$ y el citerio de partición $\theta=(j,t_m)$ sobre la característica $j$ y el valor de corte $t_m$ que divide el nodo $Q$ en los dos subconjuntos:

- $Q_{izquierda}(\theta)$ definido como $\{x \in Q / x_j \le t_m\}$. 
- $Q_{derecha}(\theta)$ definido como $\{x \in Q / x_j > t_m\}$.

La valoración de criterio  $\theta$ en el nodo $Q$ se calcula como:
- $G(Q,\theta)=\frac{n_{izquierdo}}{Q} H(Q_{izquierdo})+ \frac{n_{derecho}}{Q} H(Q_{derecho})$.

Se selecciona el parámtero $\theta^*$ pare que $G(\theta^*,Q)$ sea mínimo.

De forma recursiva se vuelve a subdivir $Q_{izquierdo}$ y $Q_{derecho}$ hasta alcanzar unos conjuntos unitarios o el criterio para subdividir no se cumpla.

# Clasificación $H(Q)$:
Evaluación de la uniformidad de un nodo.

Consideramos la probabilidad de la clase k en el nodo $Q_m$ es $p_{m,k}=\frac{1}{N_m} \sum_{x_i \in Q_m} I(y_i=k)$.


|**Criterio**       |   **Valor**                                              |
|-------------------|----------------------------------------------------------|
|**Gini**           |$H(Q_m)=\sum_k p_{m,k}(1-p_{m,k})$                        |
|-------------------|----------------------------------------------------------|
|**Entropía**       |$H(Q_m)=-\sum_k p_{m,k}log(p_{m,k})$                      |
|-------------------|----------------------------------------------------------|
|**Mal clasificación**|$H(Q_m)=1-max(p_{m,k})$                                 |
|-------------------|----------------------------------------------------------|

- Gini: mide como confluye el acierto y el error. Me da valores altos cuando la clase no es tan clara. El valor máximo es cuando uno de ellos vale 0 porque el valor resultante es 1. Mi objetivo es obtener un valor de gini pequeño.
- Entropía: desorden. Probabilidad de encontrarlo. Cuánto más probabilidades sean 0, más cercano a 1. 
- Mal clasificación: la inversa del máx de la clase que mejor clasifica.


# Ejemplo de divisiones $\theta$:
Datos de flores Iris.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_iris
#leemos los datos
iris = load_iris()

Creamos el DataFrame  **datos** correspondientes a partir de la librería **sklearn**

|                 | Características|                 |                |       |
|-----------------|----------------|-----------------|----------------|-------|
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)| tipos |



|Tipos|Clase      |
|-----|-----------|
0    |setosa
1    |versicolor
2    |virginia


In [ ]:
datos=pd.DataFrame(iris.data, columns=iris.feature_names)
datos['tipos']=pd.Series(iris.target)
datos.head()

## Probar una división sobre la característica **sepal length (cm)**

Ver como se distribuyen los valores de la característica

In [ ]:
caracteristica=iris.feature_names[0] # Columna 0 'sepal lenght (cm)'
datos[caracteristica].hist()

Utilizamos el valor 6 para realizar la partición

- $Q_{izqierdo}=\{x \in datos| x[sepal\ leng (cm)] \le 6\}$
- $Q_{derecho}=\{x \in datos| x[sepal\ leng (cm)] > 6\}$

Se utiliza la función [**cut**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html) de la librería **pandas** que segmenta una serie en intervalos.

En ejemplo se proponen dos intervalo :
- Izquierdo (0,6]
- Derecho (6,8]

In [ ]:
from pandas._libs.interval import Interval #Construyo intervalos
corte=6
dv=pd.cut(datos[caracteristica],[0,corte,8]) #Doy los cortes que quiero tener. 
dv_izquierdo=datos[dv==Interval(0, corte, closed='right')] #Puedo cerrarlo o no para que el extremo se incluya o no. 
dv_derecho=datos[dv==Interval(corte, 8, closed='right')]
len(dv_izquierdo),len(dv_derecho),len(datos)

Valoramos los nodos obtenido **dv_izquierdo** y **dv_derecho**.
- Obtenemos la probabilidad de cada clase $p_{m,k}$ haciendo uso de [**groupby**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html?highlight=groupby#pandas.DataFrame.groupby) y **count** de **pandas**.

In [ ]:
#Grupo izquierda. 
Nodo=dv_izquierdo
sdv=Nodo.groupby('tipos',as_index=False).count()
nizq=len(Nodo)
sdv['Prob']=sdv[caracteristica]/nizq
lpizq=sdv['Prob'].values
sdv
#Calculamos las probabilidades. 

In [ ]:
#Grupo de la derecha. 
Nodo=dv_derecho
sdv=Nodo.groupby('tipos',as_index=False).count()
nder=len(Nodo)
sdv['Prob']=sdv[caracteristica]/nder
lpder=sdv['Prob'].values
sdv

Valoramos la partición:
- Calculamos el valor de disparidad en cada nodo: Gini, Entropía y mal clasificación.

In [ ]:
def Gini (l_probabilidades):
  s=0.0
  for p in l_probabilidades:
    s = s + p*(1-p)
  return s

def Entropia(l_probabilidades):
  s = 0.0
  log_list= np.log(l_probabilidades)
  for (p,plog) in zip(l_probabilidades,log_list):
    s= s +p *plog
  return - s

def MClasificacion(l_probabilidades):
  return 1-max(l_probabilidades)

  # in zip -> de dos o tres listas te genera una lista donde cada uno de los elementos de la lista es una tupla. Es decir: [1,2][a,b]->[1,a][2,b].


Valoración utilizando el criterio **gini**

In [ ]:
print(f'Gini(lpizq)={Gini(lpizq)}\nGini(lpder)={Gini(lpder)}\nG(izq,(sepal,6))={(nizq/(nizq+nder))*Gini(lpizq)+(nder/(nizq+nder))*Gini(lpder)}')

Valoración utilizando la **entropía**

In [ ]:
print(f'Entropia(lpizq)={Entropia(lpizq)}\nEntropia(lpder)={Entropia(lpder)}\nG(izq,(sepal,6))={(nizq/(nizq+nder))*Entropia(lpizq)+(nder/(nizq+nder))*Entropia(lpder)}')

Valoración utilizando la **mal clasificación**

In [ ]:
print(f'Entropia(lpizq)={MClasificacion(lpizq)}\nEntropia(lpder)={MClasificacion(lpder)}\nG(izq,(sepal,6))={(nizq/(nizq+nder))*MClasificacion(lpizq)+(nder/(nizq+nder))*MClasificacion(lpder)}')

# Scikit-Learn [Árboles de decisión](https://scikit-learn.org/stable/modules/tree.html).

**Árboles de clasificación** \( [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)\).


- **Leemos** los datos.
- **Importamos** toda la funcionalidad de los árboles \(**tree**\).
- **Creamos** el árbol de decisión para clasificar.
- Lo instanciamos \(**aprendemos**\) con los datos **X** y sus clase **y**.

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
X, y = load_iris(return_X_y=True)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y) #Línea de ajuste. Como estamos en un ajuste supervisado necesito la x y la y para hacer el ajuste. Antes sólo la x y las clases no (y). 

Mostramos el árbol obtenido.

In [ ]:
tree.plot_tree(clf)

Predecimos como con cualquier modelo .

In [ ]:
clf.predict(X)!=y

Si hay algo distinto o algo raro me daría algún True. 

Podemos exporta el gráfico del árbol aun fichero **pdf** con el uso de la librería [**graphviz**](https://pypi.org/project/graphviz/).

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris")

La exportación puede ser más *sofisticada* y a distintos *formatos*.

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                     feature_names=iris.feature_names,  
                     class_names=iris.target_names,  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)
graph

La anchura del pétalo es menos importante que la longitud... y así vamos viendo los resultados. Los colores indican la complejidad, el más complejo es el blanco que es la combinación de todos. 

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
iris = load_iris()
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree = decision_tree.fit(iris.data, iris.target)
r = export_text(decision_tree, feature_names=iris['feature_names'])
print(r)

# [**Orange**](https://orange.biolab.si/) un entorno integrado **Open Source**
Como hacernos la vida un poco más fácil.